In [2]:
# Install Julia from https://julialang.org/

# Install JWAS from https://reworkhow.github.io/JWAS.jl/latest/


In [ ]:
# Single-trait analysis - https://github.com/reworkhow/JWAS.jl/wiki/Single-Trait-Analysis

In [ ]:
# Step 1: Load packages

In [37]:
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets

In [38]:
# Step 2: Read data 

In [39]:
phenofile  = dataset("phenotypes.csv")
pedfile    = dataset("pedigree.csv")
genofile   = dataset("genotypes.csv")

"/Users/gota/.julia/packages/JWAS/6mjQ8/src/4.Datasets/src/../data/genotypes.csv"

In [40]:
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");

The delimiter in pedigree.csv is ','.
Pedigree information:
#individuals: 100
#sires:       39
#dams:        38
#founders:    20
The delimiterd in genotypes.csv is ','. The header (marker IDs) is provided in genotypes.csv.
Genotype informatin:
#markers: 1000; #individuals: 60


In [41]:
first(phenotypes,5)

,ID,y1,y2,y3,x1,x2,x3,dam,bv1
,String,Float64,Float64,Float64,Float64,String,String,String?,Float64
1,a1,-0.579682,-0.0582089,1.0456,0.77,g1,m,missing,-1.07018
2,a2,-2.02246,-2.3843,-1.57196,-1.02,g2,m,missing,-1.03328
3,a3,-1.48071,-0.421258,-0.272245,0.52,g1,f,missing,-1.2916
4,a4,-3.03031,-2.0634,-0.604634,-1.05,g4,m,missing,-1.39308
5,a5,2.18819,2.22425,1.72306,2.06,g3,m,missing,0.267546


In [42]:
# Step 3: Build Model Equations

In [43]:
model_equation  ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes"

"y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes"

In [44]:
model = build_model(model_equation);

In [45]:
# Step 4: Set Factors or Covariates

In [46]:
set_covariate(model,"x1");

In [47]:
# Step 5: Set Random or Fixed Effects

In [48]:
set_random(model,"x2");
# set_random(model,"ID dam",pedigree);

In [49]:
# # Step 6: Run Analysis

In [50]:
out=runMCMC(model,phenotypes);

The folder results is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
In this complete genomic data (non-single-step) analyis, 40 phenotyped individuals are not genotyped. These are removed from the analysis.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Phenotypes for 60 observations are used in the analysis.These individual IDs are saved in the file IDs_for_individuals_with_phenotypes.txt.
Prior information for genomic variance is not provided and is generated from the data.
Prior information for residual variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.

The prior for marker effects variance is calculated from the genetic variance and π.
The mean of the prior for the marker effects variance

running MCMC ...100%|███████████████████████████████████| Time: 0:00:00




The version of Julia and Platform in use:

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




In [51]:
# Check the results

In [52]:
out

Dict{Any, Any} with 7 entries:
  "EBV_y1"                   => 60×3 DataFrame…
  "heritability"             => 1×3 DataFrame…
  "location parameters"      => 106×5 DataFrame…
  "residual variance"        => 1×3 DataFrame…
  "pi_genotypes"             => 1×3 DataFrame…
  "genetic_variance"         => 1×3 DataFrame…
  "marker effects genotypes" => 1000×5 DataFrame…

In [53]:
keys(out)

KeySet for a Dict{Any, Any} with 7 entries. Keys:
  "EBV_y1"
  "heritability"
  "location parameters"
  "residual variance"
  "pi_genotypes"
  "genetic_variance"
  "marker effects genotypes"

In [54]:
out["heritability"]

,Covariance,Estimate,SD
,Any,Any,Any
1,y1,0.674308,0.129871


In [55]:
out["residual variance"]

,Covariance,Estimate,SD
,Any,Any,Any
1,y1_y1,0.419852,0.176563


In [56]:
out["marker effects genotypes"]

,Trait,Marker_ID,Estimate,SD,Model_Frequency
,Any,Any,Any,Any,Any
1,y1,m1,-0.00263039,0.0597802,0.97
2,y1,m2,-0.000664603,0.0614825,0.99
3,y1,m3,-0.000599556,0.0577818,0.99
4,y1,m4,0.0066331,0.0603599,0.99
5,y1,m5,0.000589284,0.0539395,0.99
6,y1,m6,0.00111894,0.0555943,0.96
7,y1,m7,-0.00871495,0.0512356,1.0
8,y1,m8,-0.00392768,0.0584464,1.0
9,y1,m9,0.0147966,0.0619313,0.96


In [57]:
# Bayesian marker effect SEM-GWAS

In [58]:
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets

In [59]:
# Step 2: Read data 
phenofile  = dataset("phenotypes.csv")
pedfile    = dataset("pedigree.csv")
genofile   = dataset("genotypes.csv")
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");
first(phenotypes,5)

The delimiter in pedigree.csv is ','.
Pedigree information:
#individuals: 100
#sires:       39
#dams:        38
#founders:    20
The delimiterd in genotypes.csv is ','. The header (marker IDs) is provided in genotypes.csv.
Genotype informatin:
#markers: 1000; #individuals: 60


,ID,y1,y2,y3,x1,x2,x3,dam,bv1
,String,Float64,Float64,Float64,Float64,String,String,String?,Float64
1,a1,-0.579682,-0.0582089,1.0456,0.77,g1,m,missing,-1.07018
2,a2,-2.02246,-2.3843,-1.57196,-1.02,g2,m,missing,-1.03328
3,a3,-1.48071,-0.421258,-0.272245,0.52,g1,f,missing,-1.2916
4,a4,-3.03031,-2.0634,-0.604634,-1.05,g4,m,missing,-1.39308
5,a5,2.18819,2.22425,1.72306,2.06,g3,m,missing,0.267546


In [60]:
# Step 3: Build Model Equations

model_equation  ="y1 = intercept + x1 + x2 + x2*x3 + ID + dam + genotypes
                  y2 = intercept + x1 + x2 + ID + genotypes
                  y3 = intercept + x1 + ID + genotypes";
model = build_model(model_equation);

In [61]:
# Step 4: Set Factors or Covariates
set_covariate(model,"x1");

In [62]:
# Step 5: Set Random or Fixed Effects
set_random(model,"x2");
set_random(model,"ID dam",pedigree);

x2 is not found in model equation 3.
dam is not found in model equation 2.
dam is not found in model equation 3.


In [63]:
# Step 6: Run Analysis
# If `causal_structure` is provided, e.g., causal_structure = [0.0 0.0 0.0;1.0 0.0 0.0;1.0 0.0 0.0] (column index affects row index, and a lower triangular matrix is required) for
# trait 1 -> trait 2 and trait 1 -> trait 3, phenotypic causal networks will be incorporated using structure equation models.
my_structure = [0.0 0.0 0.0
                1.0 0.0 0.0
                1.0 0.0 0.0]
out=runMCMC(model,phenotypes,causal_structure=my_structure);

The folder results already exists.
The folder results1 is created to save results.
Checking pedigree...
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
In this complete genomic data (non-single-step) analyis, 40 phenotyped individuals are not genotyped. These are removed from the analysis.
Predicted values for individuals of interest will be obtained as the summation of Any["y1:ID", "y2:ID", "y3:ID", "y1:dam"] (Note that genomic data is always included for now).Phenotypes for 60 observations are used in the analysis.These individual IDs are saved in the file IDs_for_individuals_with_phenotypes.txt.
Prior information for genomic variance is not provided and is generated from the data.
Prior information for residual variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
Prior information for random effect variance is

running MCMC ...100%|███████████████████████████████████| Time: 0:00:02




The version of Julia and Platform in use:

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.


Compute the model frequency for each marker (the probability the marker is included in the model).
Compute the model frequency for each marker (the probability the marker is included in the model).
Compute the model frequency for each marker (the probability the marker is included in the model).
Compute the model frequency for each marker (the probability the marker is included in the model).
Compute the model frequency for each marker (the probability the marker is included in the model).
Compute the model frequency for each marker (the probability the marker is included in the 

In [64]:
# GWAS of Direct Marker Effects on Trait y2

#Compute the model frequency for each marker (the probability the marker is included in the model).
marker_effects_file="results1/MCMC_samples_marker_effects_genotypes_y2.txt"
out = GWAS(marker_effects_file,header=true)



Compute the model frequency for each marker (the probability the marker is included in the model).


,marker_ID,modelfrequency
,Abstrac…,Float64
1,m1,0.98
2,m2,0.95
3,m3,0.98
4,m4,0.96
5,m5,0.92
6,m6,1.0
7,m7,0.98
8,m8,0.97
9,m9,0.95


In [65]:
# GWAS of Indirect Marker Effects on Trait y2

#Compute the model frequency for each marker (the probability the marker is included in the model).
marker_effects_file="results1/MCMC_samples_indirect_marker_effects_genotypes_y2.txt"
out=GWAS(marker_effects_file,header=true)



Compute the model frequency for each marker (the probability the marker is included in the model).


,marker_ID,modelfrequency
,Abstrac…,Float64
1,m1,0.96
2,m2,0.99
3,m3,1.0
4,m4,0.92
5,m5,0.97
6,m6,1.0
7,m7,0.93
8,m8,0.96
9,m9,0.96
